In [ ]:
from tensorflow.keras.models import Model # Импортируем модели keras: Model
from tensorflow.keras.layers import * # Импортируем все слои keras
from tensorflow.keras import backend as K # Импортируем модуль backend keras'а
from tensorflow.keras.optimizers import Adam # Импортируем оптимизатор Adam
from tensorflow.keras import utils # Импортируем модуль utils библиотеки tensorflow.keras для получения OHE-представления
import tensorflow as tf
from google.colab import files # Импортируем Модуль files для работы с файлами
import matplotlib.pyplot as plt # Импортируем модуль pyplot библиотеки matplotlib для построения графиков
from tensorflow.keras.preprocessing import image # Импортируем модуль image для работы с изображениями
import numpy as np # Импортируем библиотеку numpy
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import *
from tensorflow.keras.layers.experimental.preprocessing import Resizing
import time
import random
import shutil
import os # Импортируем библиотеку os для раоты с фаловой системой
from PIL import Image # импортируем модель Image для работы с изображениями
import seaborn as sns

import numba as nb
import cv2 as cv
import json
from PIL import Image, ImageDraw
from numpy.lib.stride_tricks import as_strided
sns.set_style('darkgrid')

In [ ]:
from google.colab import drive # Подключаем гугл-диск
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
directory = '/content/drive/My Drive/segmentation/' # Указываем путь к обучающей выборке с оригинальными изображения

# Загружаем и обрабатываем данные

In [ ]:
with open(directory + "via_POLIGON_data.json", "r") as write_file:
    data = json.load(write_file)

In [ ]:
images = []
for filename in data.values():
    temp_img = np.array(Image.open(directory + 'orig/' + 'дог' + str(int(filename['filename'][0]) + 1) + '.jpg'))
    temp_img = cv.resize(temp_img, (768, 512), interpolation = cv.INTER_NEAREST)
    images.append(temp_img)


images = np.array(images)

In [ ]:
all_points = []

for image_data in data.values():
  point = image_data['regions']['0']['shape_attributes']
  four_coords = tuple(zip(point['all_points_x'], point['all_points_y']))
  all_points.append(four_coords)

all_points = np.array(all_points)[:, 0:4, :].astype(np.float32)
all_points[:, :, 0] = all_points[:, :, 0]/(images.shape[2] * 2)
all_points[:, :, 1] = all_points[:, :, 1]/(images.shape[1] * 2)
all_points = all_points.reshape(-1, 8)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(images, all_points, test_size = 0.2)

# Создаем Собственную Нейросеть

In [ ]:
def reslayer(*args, x, n = 5, **kwargs):
  y = Conv2D(padding = 'same', activation = 'relu', *args, **kwargs)(x)
  for i in range(n):
    y = Conv2D(padding = 'same', activation = 'relu', *args, **kwargs)(y)
    y = BatchNormalization()(y)
  y = Add()([x, y])
  return y

In [ ]:
input_layer = Input((x_train.shape[1], x_train.shape[2], x_train.shape[3]))
Conv_1 = Conv2D(32, (2, 2), padding = 'same', activation = 'relu')(input_layer)

Conv_2 = Conv2D(64, (2, 2), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_1)
Conv_2 = reslayer(64, (2, 2), x = Conv_2)

Conv_3 = Conv2D(96, (2, 2), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_2)
Conv_3 = reslayer(96, (2, 2), x = Conv_3)

Conv_4 = Conv2D(128, (2, 2), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_3)
Conv_4 = reslayer(128, (2, 2), x = Conv_4)

Conv_5 = Conv2D(160, (2, 2), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_4)
Conv_5 = reslayer(160, (2, 2), x = Conv_5)

Conv_6 = Conv2D(180, (2, 2), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_5)
Conv_6 = reslayer(180, (2, 2), x = Conv_6)

Conv_7 = Conv2D(200, (2, 2), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_6)
Conv_7 = reslayer(200, (2, 2), x = Conv_7)

Conv_8 = Conv2D(256, (2, 2), strides = (2, 2), padding = 'same', activation = 'relu')(Conv_7)
Conv_8 = reslayer(256, (2, 2), x = Conv_8)

flatten_layer = Flatten()(Conv_8)
dense1 = Dense(1024, activation = 'relu')(flatten_layer)
dense1 = Dropout(0.2)(dense1)
dense1 = BatchNormalization()(dense1)
dense2 = Dense(512, activation = 'relu')(dense1)
dense2 = Dropout(0.2)(dense2)
dense2 = BatchNormalization()(dense2)

output_layer = Dense(8, activation = 'sigmoid')(dense2)

model = Model(input_layer, output_layer)

model.compile(optimizer=Adam(lr = 0.001), loss='binary_crossentropy', metrics=['mae'])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 768, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 768, 32) 416         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 384, 64) 8256        conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 384, 64) 16448       conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
cb1 = ModelCheckpoint(
    '/content/drive/My Drive/segmentation/weightsRegressor/',
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="min",
    save_freq="epoch",
    options=None
)

cb2 = ReduceLROnPlateau()

history = model.fit(x_train/255, y_train, epochs=150, batch_size=16, validation_data = (x_test/255, y_test), callbacks = [cb1, cb2]) #  Обучаем модель на выборке по трем классам на полноразмерных изображениях

Epoch 1/150
6/6 [==============================] - 30s 2s/step - loss: 0.8890 - mae: 0.4569 - val_loss: 0.6369 - val_mae: 0.4283

Epoch 00001: val_loss improved from inf to 0.63685, saving model to /content/drive/My Drive/segmentation/weightsRegressor/
Epoch 2/150
6/6 [==============================] - 8s 1s/step - loss: 0.8583 - mae: 0.4418 - val_loss: 0.5951 - val_mae: 0.4027

Epoch 00002: val_loss improved from 0.63685 to 0.59507, saving model to /content/drive/My Drive/segmentation/weightsRegressor/
Epoch 3/150
6/6 [==============================] - 8s 1s/step - loss: 0.8264 - mae: 0.4362 - val_loss: 0.5736 - val_mae: 0.3888

Epoch 00003: val_loss improved from 0.59507 to 0.57361, saving model to /content/drive/My Drive/segmentation/weightsRegressor/
Epoch 4/150
6/6 [==============================] - 8s 1s/step - loss: 0.7952 - mae: 0.4289 - val_loss: 0.5495 - val_mae: 0.3738

Epoch 00004: val_loss improved from 0.57361 to 0.54948, saving model to /content/drive/My Drive/segmentati

In [ ]:
input_layer = Input((x_train.shape[1], x_train.shape[2], x_train.shape[3]))
resize_layer = Resizing(224, 224)(input_layer)

resnet = VGG19()(resize_layer)

x = Dropout(0.1)(resnet)
out = Dense(8, activation = 'sigmoid')(x)

model = Model(input_layer, out)

model.compile(optimizer=Adam(lr = 0.001), loss='binary_crossentropy', metrics=['mae'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512, 768, 3)]     0         
_________________________________________________________________
resizing (Resizing)          (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 1000)              143667240 
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 8008      
Total params: 143,675,248
Trainable params: 143,675,248
Non-trainable params: 0
_________________________________________________________________


# Пробуем Предобученную Сетку

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

cb = ReduceLROnPlateau()

history = model.fit(x_train/255, y_train, epochs=250, batch_size=8, validation_data = (x_test/255, y_test), callbacks = [cb]) #  Обучаем модель на выборке по трем классам на полноразмерных изображениях

Epoch 1/250
11/11 [==============================] - 43s 632ms/step - loss: 0.6846 - mae: 0.4547 - val_loss: 0.6749 - val_mae: 0.4483
Epoch 2/250
11/11 [==============================] - 2s 209ms/step - loss: 0.6725 - mae: 0.4484 - val_loss: 0.6651 - val_mae: 0.4428
Epoch 3/250
11/11 [==============================] - 2s 207ms/step - loss: 0.6640 - mae: 0.4386 - val_loss: 0.6555 - val_mae: 0.4374
Epoch 4/250
11/11 [==============================] - 2s 209ms/step - loss: 0.6533 - mae: 0.4326 - val_loss: 0.6461 - val_mae: 0.4320
Epoch 5/250
11/11 [==============================] - 2s 208ms/step - loss: 0.6424 - mae: 0.4335 - val_loss: 0.6369 - val_mae: 0.4266
Epoch 6/250
11/11 [==============================] - 2s 210ms/step - loss: 0.6374 - mae: 0.4241 - val_loss: 0.6279 - val_mae: 0.4214
Epoch 7/250
11/11 [==============================] - 2s 213ms/step - loss: 0.6277 - mae: 0.4246 - val_loss: 0.6190 - val_mae: 0.4161
Epoch 8/250
11/11 [==============================] - 2s 214ms/step -

In [ ]:
history = model.fit(x_train/255, y_train, epochs=250, batch_size=8, validation_data = (x_test/255, y_test), callbacks = [cb]) #  Обучаем модель на выборке по трем классам на полноразмерных изображениях

Epoch 1/250
11/11 [==============================] - 2s 212ms/step - loss: 0.1891 - mae: 0.0528 - val_loss: 0.1776 - val_mae: 0.0399
Epoch 2/250
11/11 [==============================] - 2s 208ms/step - loss: 0.1894 - mae: 0.0530 - val_loss: 0.1775 - val_mae: 0.0398
Epoch 3/250
11/11 [==============================] - 2s 209ms/step - loss: 0.1842 - mae: 0.0478 - val_loss: 0.1774 - val_mae: 0.0397
Epoch 4/250
11/11 [==============================] - 2s 211ms/step - loss: 0.1858 - mae: 0.0488 - val_loss: 0.1774 - val_mae: 0.0395
Epoch 5/250
11/11 [==============================] - 2s 211ms/step - loss: 0.1831 - mae: 0.0453 - val_loss: 0.1773 - val_mae: 0.0394
Epoch 6/250
11/11 [==============================] - 2s 211ms/step - loss: 0.1824 - mae: 0.0441 - val_loss: 0.1772 - val_mae: 0.0394
Epoch 7/250
11/11 [==============================] - 2s 211ms/step - loss: 0.1854 - mae: 0.0486 - val_loss: 0.1771 - val_mae: 0.0393
Epoch 8/250
11/11 [==============================] - 2s 212ms/step - 

In [ ]:
model.save(directory + '/weightsRegressor/vgg_model_pretrained.h5')

In [ ]:
def gather_four_coords(points):
  results = [(round(points[i] * x_train.shape[2]), round(points[i + 1] * x_train.shape[1])) for i in range(0, len(points), 2)] + [(round(points[0] * x_train.shape[2]), round(points[1] * x_train.shape[1]))]
  return results

predicted_coords = model.predict(x_test/255, batch_size = 2, verbose = 0)
for i, image in enumerate(x_test):
  image = Image.fromarray(image)
  points = gather_four_coords(predicted_coords[i])
  draw = ImageDraw.Draw(image)
  draw.line(points, width = 5, fill="yellow")
  image.save(directory + f"regressor_predicted/vgg_results/{i}.jpg")

Итоги:
* Нужно обучать на большом количестве эпох
* resnet не даёт более хорошие результаты
* Dropout в свёрточных слоях ухудшает результаты
* binary_crossentropy быстрее обучается